In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
def prepare_data(file_path):
    df = pd.read_csv(file_path)
    return df

In [ ]:
def engineer_features(df):
    df = pd.get_dummies(df, columns=['State', 'Crop', 'Season'])
    X = df.drop('Yield', axis=1)
    y = df['Yield']
    
    return X, y

In [ ]:
def create_model(input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_dim=input_dim),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1)
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model



In [ ]:
def train_model(model, X_train, y_train, X_val, y_val, epochs=500, batch_size=32):
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        verbose=1
    )
    return history



In [ ]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"Mean Squared Error: {mse}")
    print(f"Mean Absolute Error: {mae}")
    return mse, mae



In [ ]:
if __name__ == "__main__":
    df = prepare_data("/workspaces/Crop-yield-prediction/crop_yield.csv")
    X, y = engineer_features(df)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)
    
    model = create_model(X_train.shape[1])
    history = train_model(model, X_train_scaled, y_train, X_val_scaled, y_val)
    
    mse, mae = evaluate_model(model, X_test_scaled, y_test)
    
    model.save('crop_yield_prediction_model.h5')

    print("Model training complete and saved.")